# Analyse Reference catalog

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/03/04
- update : 2024/03/04


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import math
import lsst.geom

In [ ]:
ls ../data/202402

In [ ]:
filesource_in="../data/202402/atlas_refcat2_20220201_202402.csv"

In [ ]:
# Load data (deserialize)
import pickle
#with open('Latiss_tractspatchs.pickle', 'rb') as handle:
with open('../data/202402/Latiss_tractspatchs_radec_htm7_skypixels_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

# Read input catalog

In [ ]:
df = pd.read_csv(filesource_in,index_col=0)

In [ ]:
df

In [ ]:
colnames = list(df.columns)
for name in colnames:
    print(name)

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
TRACT_SEL = 3864


List_Skypixels_selected = tracts_patchs_dict[TRACT_SEL]['htmpixels']
RAMIN,RAMAX,DECMIN,DECMAX = tracts_patchs_dict[TRACT_SEL]['radec']
NPIX = len(List_Skypixels_selected)

if NPIX>0:
    cut = (df.skypix == List_Skypixels_selected[0])
    
if NPIX == 2:
    cut = cut | (df.skypix == List_Skypixels_selected[1])
    
df = df[cut]

In [ ]:
df

In [ ]:
df["ra"] = df["coord_ra"]*180./np.pi
df["dec"] = df["coord_dec"]*180./np.pi

In [ ]:
df

In [ ]:
df["g_mag"] = df["g_flux"].apply(convert_fluxtomag)
df["r_mag"] = df["r_flux"].apply(convert_fluxtomag)
df["i_mag"] = df["i_flux"].apply(convert_fluxtomag)
df["z_mag"] = df["z_flux"].apply(convert_fluxtomag)

In [ ]:
variable_names = ["g_mag","r_mag","i_mag","z_mag"]
all_bands = ["g","r","i","z"]
all_band_colors = ["g","r","orange","grey"]
NB = len(all_bands)

fig,axs  = plt.subplots(nrows=1,ncols=NB,figsize=(16,4))

for iband in range(NB):
    name = variable_names[iband]
    df[name].hist(bins=50,range=(8,22),ax=axs[iband],facecolor=all_band_colors[iband],legend=True)
    #ax.legend()
plt.suptitle(filesource_in)

In [ ]:
cmap = matplotlib.cm.inferno
cmap_reversed = matplotlib.cm.get_cmap('inferno_r')


In [ ]:
xvals = [RAMIN,RAMAX,RAMAX,RAMIN,RAMIN]
yvals = [DECMIN,DECMIN,DECMAX,DECMAX,DECMIN]

In [ ]:
MAGCUT = 12

cut = (df[variable_names[0] ] < MAGCUT)  & (df[variable_names[1] ] < MAGCUT) & (df[variable_names[2] ] < MAGCUT) & (df[variable_names[0] ] < MAGCUT)

df_cut = df[cut]
my_colors = df_cut["i_mag"].values
fig, ax = plt.subplots(ncols=1,nrows=1,figsize=(8,8))

im = df_cut.plot.scatter('ra','dec',ax=ax,marker='o',color=my_colors,cmap="jet_r")
ax.plot(xvals,yvals,color='k')
ax.grid()

#cbar = plt.colorbar(ax.collections[0], ax=ax)
plt.gca().set_aspect('equal')
title = f"catalog sources m_I<{MAGCUT} mag in tract {TRACT_SEL}"
plt.title(title)